In [1]:
import re, os, errno

In [16]:
def preprocesar(ruta_dir, archivos):
    try:
        ruta_folder = os.path.join(ruta_abs, 'preprocesados')
        os.makedirs(ruta_folder) # cre
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    remplazos = [
        # Remplazo dobles lineas por un identificador especial
        (r'\n\n', '<dnl>'),
        # Remplazo lineas simples por espacios
        (r'\n', ' '),
        # Remplazo identificador de linea doble por linea simple
        (r'<dnl>', '\n'),
        # Remplazo caracter &nbsp (\u00A0) por espacio
        (r'\u00A0', ' '),
        # Remplazo tabulador por espacio
        (r'\t', ' '),
        # Remplazo repeticiones de más de dos espacios por espacio simple
        (r' ( )+', ' '),
        # Remplaza cadenas encerradas dentro de doble guión bajo,
        # por ejemplo: _Hola_, _cómo estás_
        (r'_(?P<word>[^_]+)_', r'\g<word>'),
        # Elimina links a imágenes tipo [C:\Hola\Mundo.jpg]
        (r'\[(.+)+\]', '')
    ]
    for fname in archivos:
        contenido = ""
        full_path = os.path.join(ruta_abs, fname)
#         print('Procesando: ', full_path, ' ...')
        with open(full_path, 'r', encoding='utf-8') as f:
            contenido = f.read()
        for viejo, nuevo in remplazos:
            contenido = re.sub(viejo, nuevo, contenido)
            with open(os.path.join(ruta_folder, fname), 'w', encoding='utf-8') as wf:
                wf.write(contenido)

In [17]:
for candidato in ['AMLO', 'JAMK', 'RAC']:
    ruta_rel = os.path.join('..', 'Boletines', candidato, 'txt')
    ruta_abs = os.path.abspath(ruta_rel)
    # Quitamos los directorios
    archivos = [
        f for f in os.listdir(ruta_abs) 
        if os.path.isfile(os.path.join(ruta_abs, f))
    ] 
#     print(ruta_abs)
    preprocesar(ruta_abs, archivos)